In [287]:
# import all
import pandas as pd
import numpy as np

In [ ]:
'''
Read the emails for the json and process it for data viz
'''

In [288]:
email_json = "C:/Users/ghosh/Google Drive/personal_project/Learning_NLP/Gmail_analysis/all_gmails_output.json"

In [289]:
# read the json
email_data = pd.read_json(email_json)

In [292]:
email_data = email_data[['From', 'To', 'Date', 'Subject']]

#email_data.drop_duplicates() ## Error unhashable type: 'list'
# duplicating date column for further processing
email_data['Date_new'] = email_data['Date']

In [293]:
email_data

,From,To,Date,Subject,Date_new
0,admin@anandrathi.com,[GHOSHM21@GMAIL.COM],Fri Sep 14 13:44:44 2007 GMT,User Successfully Added,Fri Sep 14 13:44:44 2007 GMT
1,crcedp@rathi.com,[GHOSHM21@GMAIL.COM],"Mon, 5 Nov 2007 22:25:41 +0530",BSECAPITAL Ledger,"Mon, 5 Nov 2007 22:25:41 +0530"
2,ishasaraiya@rathi.com,[ishasaraiya@rathi.com],"Mon, 5 May 2008 18:51:57 +0530",Minimum Requirements,"Mon, 5 May 2008 18:51:57 +0530"
3,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 16:03:04 +0530",DSL Welcome Letter,"Tue, 15 Feb 2011 16:03:04 +0530"
4,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 15:59:35 +0530",DSL Welcome Letter,"Tue, 15 Feb 2011 15:59:35 +0530"
...,...,...,...,...,...
102234,sandipag@anz.com,[ghoshm21@gmail.com],"Fri, 2 Mar 2007 20:31:08 +0530",,"Fri, 2 Mar 2007 20:31:08 +0530"
102235,sandipag@anz.com,[ghoshm21@gmail.com],"Fri, 2 Mar 2007 20:30:04 +0530",FW: ss,"Fri, 2 Mar 2007 20:30:04 +0530"
102236,sandipag@anz.com,[ghoshm21@gmail.com],"Fri, 2 Mar 2007 20:31:41 +0530",,"Fri, 2 Mar 2007 20:31:41 +0530"
102237,sandipag@anz.com,[ghoshm21@gmail.com],"Tue, 20 Mar 2007 14:04:47 +0530",FW: Good ppt on Data warehousing / Data mining,"Tue, 20 Mar 2007 14:04:47 +0530"


In [294]:
# clean and conver all date columns to proper IST date column then get date and time part
import re
# Thu,  1 Nov 2007 17:24:14 -0300 (GMT+3)
# remove the extra GMT and o'clock GMT. These encounter error while using in pandas date
# these are not valid date time in pandas
email_data.Date_new = email_data.Date_new.str.replace(r'(\(GMT.*\))', '', regex = True)
email_data.Date_new = email_data.Date_new.str.replace(r"(o'clock GMT.*)", "GMT", regex = True)
# convert to utc and if the format do not match then keep it null
email_data['Datetime_utc'] = pd.to_datetime(email_data.Date_new, utc=True, errors='coerce')
# convert to IST and get date and time
email_data['datetime_ist'] = email_data.Datetime_utc.dt.tz_convert('Asia/Kolkata')
email_data['only_Date'] = email_data['datetime_ist'].dt.strftime('%Y-%m-%d')
email_data['only_time'] = email_data['datetime_ist'].dt.strftime('%H:%M:%S')
# drop columns
email_data = email_data.drop(['Date', 'Date_new', 'Datetime_utc'], axis=1)
email_data

,From,To,Subject,datetime_ist,only_Date,only_time
0,admin@anandrathi.com,[GHOSHM21@GMAIL.COM],User Successfully Added,2007-09-14 19:14:44+05:30,2007-09-14,19:14:44
1,crcedp@rathi.com,[GHOSHM21@GMAIL.COM],BSECAPITAL Ledger,2007-11-05 22:25:41+05:30,2007-11-05,22:25:41
2,ishasaraiya@rathi.com,[ishasaraiya@rathi.com],Minimum Requirements,2008-05-05 18:51:57+05:30,2008-05-05,18:51:57
3,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],DSL Welcome Letter,2011-02-15 16:03:04+05:30,2011-02-15,16:03:04
4,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],DSL Welcome Letter,2011-02-15 15:59:35+05:30,2011-02-15,15:59:35
...,...,...,...,...,...,...
102234,sandipag@anz.com,[ghoshm21@gmail.com],,2007-03-02 20:31:08+05:30,2007-03-02,20:31:08
102235,sandipag@anz.com,[ghoshm21@gmail.com],FW: ss,2007-03-02 20:30:04+05:30,2007-03-02,20:30:04
102236,sandipag@anz.com,[ghoshm21@gmail.com],,2007-03-02 20:31:41+05:30,2007-03-02,20:31:41
102237,sandipag@anz.com,[ghoshm21@gmail.com],FW: Good ppt on Data warehousing / Data mining,2007-03-20 14:04:47+05:30,2007-03-20,14:04:47


In [295]:
# split the From part to domain name 
# email_data[['domain']] = email_data.From.apply(lambda x: pd.Series(str(x).split("@")[0])) # error if no @ present
# this is better solution, as this do not produce error if @ does not present
email_data['domain'] = email_data['From'].str.split('@').str[-1]
email_data


,From,To,Subject,datetime_ist,only_Date,only_time,domain
0,admin@anandrathi.com,[GHOSHM21@GMAIL.COM],User Successfully Added,2007-09-14 19:14:44+05:30,2007-09-14,19:14:44,anandrathi.com
1,crcedp@rathi.com,[GHOSHM21@GMAIL.COM],BSECAPITAL Ledger,2007-11-05 22:25:41+05:30,2007-11-05,22:25:41,rathi.com
2,ishasaraiya@rathi.com,[ishasaraiya@rathi.com],Minimum Requirements,2008-05-05 18:51:57+05:30,2008-05-05,18:51:57,rathi.com
3,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],DSL Welcome Letter,2011-02-15 16:03:04+05:30,2011-02-15,16:03:04,airtel.in
4,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],DSL Welcome Letter,2011-02-15 15:59:35+05:30,2011-02-15,15:59:35,airtel.in
...,...,...,...,...,...,...,...
102234,sandipag@anz.com,[ghoshm21@gmail.com],,2007-03-02 20:31:08+05:30,2007-03-02,20:31:08,anz.com
102235,sandipag@anz.com,[ghoshm21@gmail.com],FW: ss,2007-03-02 20:30:04+05:30,2007-03-02,20:30:04,anz.com
102236,sandipag@anz.com,[ghoshm21@gmail.com],,2007-03-02 20:31:41+05:30,2007-03-02,20:31:41,anz.com
102237,sandipag@anz.com,[ghoshm21@gmail.com],FW: Good ppt on Data warehousing / Data mining,2007-03-20 14:04:47+05:30,2007-03-20,14:04:47,anz.com


In [312]:
# get all unique domain name for last 3 years
# i will manually flag all the safe domains. like amazon, flipkart etc.
domain_names = pd.DataFrame(email_data[(email_data.only_Date >= '2018-01-01')].domain.unique(), columns=['domain_names'])
domain_names.to_csv('E:\email_data_domain_names.xlsx')

In [321]:
# load the domains filtered data after manually flaged them and fill NaN with n
domain_category = pd.read_excel('E:\email_data_domain_names.xlsx')
domain_category.fillna('n', inplace=True)
domain_category

,index,domain_names,required
0,0,Emeritus.org,y
1,1,greatlearning.in,y
2,2,payu.in,y
3,3,nobroker.in,y
4,4,m.mail.coursera.org,y
...,...,...,...
705,705,crm.homethycare.com,n
706,706,www.luxoft.com,n
707,707,nike.com,y
708,708,talent.icims.com,y


In [327]:
# join both data on domain names
final_df = pd.merge(email_data, domain_category[['domain_names','required']], how='left', left_on='domain', right_on='domain_names').drop(['domain_names'], axis=1)
final_df.fillna('na', inplace=True)
# convert to csv, for data viz
final_df.to_csv('E:\email_data_processes.csv')

In [331]:
# count of email by year
pd.DataFrame(final_df[(final_df.only_Date >= '2019-01-01') & (final_df.only_Date <= '2019-12-31')])



,From,To,Subject,datetime_ist,only_Date,only_time,domain,required
10,gwen.scott@Emeritus.org,[ghoshm21@gmail.com],Practical Machine Learning course - UC Berkele...,2019-06-25 17:55:32+05:30,2019-06-25,17:55:32,Emeritus.org,y
14527,noreply@way2smsinvite.com,[ghoshm21@gmail.com],Reliance Jio GigaFiber Previewoffer details,2019-01-01 10:08:31+05:30,2019-01-01,10:08:31,way2smsinvite.com,n
14528,alert@icicibank.com,[ghoshm21@gmail.com],Transaction alert for your ICICI Bank debit card,2019-01-01 10:30:41+05:30,2019-01-01,10:30:41,icicibank.com,y
14529,bookings@pvrcinemas.com,[ghoshm21@gmail.com],Wishing You a Happy New Year from PVR Cinemas!,2019-01-01 10:46:14+05:30,2019-01-01,10:46:14,pvrcinemas.com,n
14530,newsletters@hdtracks.com,[ghoshm21@gmail.com],Happy New Year!,2019-01-01 10:30:31+05:30,2019-01-01,10:30:31,hdtracks.com,y
...,...,...,...,...,...,...,...,...
101579,intimations@cpc.gov.in,[ghoshm21@gmail.com],Intimation U/S 143(1) for PAN BIPxxxxx1M AY:2...,2019-09-20 08:34:25+05:30,2019-09-20,08:34:25,cpc.gov.in,n
101580,reservations@customer.goindigo.in,[ghoshm21@gmail.com],Your IndiGo Itinerary - CNLS6E,2019-10-27 13:50:55+05:30,2019-10-27,13:50:55,customer.goindigo.in,y
101581,hi@code.energy,[ghoshm21@gmail.com],Your Code Energy order is now complete,2019-11-21 08:42:03+05:30,2019-11-21,08:42:03,code.energy,n
101582,6EGSTInvoice@goindigo.in,[ghoshm21@gmail.com],TaxInvoice - WB1192011AX10874,2019-11-24 02:58:17+05:30,2019-11-24,02:58:17,goindigo.in,y


In [270]:
email_data[(email_data.From.str.contains("@")==False)].count()

From            8
To              8
Subject         8
datetime_ist    8
only_Date       8
only_time       8
domain          8
dtype: int64

In [204]:
email_data.Date = email_data.Date.str.replace(r'(\(GMT.*\))', '', regex = True)
email_data.Date = email_data.Date.str.replace(r" (o'clock GMT.*)", "", regex = True)

In [226]:
# filter (GMT+05:30)
email_data[(email_data.Date.str.contains("Sun, 13 Mar 2011 08:10:24"))]


,From,To,Date,Subject,Date_new,Datetime_utc,datetime_ist,only_Date,only_time
5,CitiAlert.India@citicorp.com,[ghoshm21@gmail.com],"Sun, 13 Mar 2011 08:10:24 +0530 (GMT+05:30)",CITIBANK BILL PAY REGISTRATION,"Sun, 13 Mar 2011 08:10:24 +0530",2011-03-13 02:40:24+00:00,2011-03-13 08:10:24+05:30,2011-03-13,08:10:24
3424,CitiAlert.India@citicorp.com,[ghoshm21@gmail.com],"Sun, 13 Mar 2011 08:10:24 +0530 (GMT+05:30)",CITIBANK BILL PAY REGISTRATION,"Sun, 13 Mar 2011 08:10:24 +0530",2011-03-13 02:40:24+00:00,2011-03-13 08:10:24+05:30,2011-03-13,08:10:24
46156,CitiAlert.India@citicorp.com,[ghoshm21@gmail.com],"Sun, 13 Mar 2011 08:10:24 +0530 (GMT+05:30)",CITIBANK BILL PAY REGISTRATION,"Sun, 13 Mar 2011 08:10:24 +0530",2011-03-13 02:40:24+00:00,2011-03-13 08:10:24+05:30,2011-03-13,08:10:24
92447,CitiAlert.India@citicorp.com,[ghoshm21@gmail.com],"Sun, 13 Mar 2011 08:10:24 +0530 (GMT+05:30)",CITIBANK BILL PAY REGISTRATION,"Sun, 13 Mar 2011 08:10:24 +0530",2011-03-13 02:40:24+00:00,2011-03-13 08:10:24+05:30,2011-03-13,08:10:24


In [227]:
email_data[(email_data.Date.str.contains("Mon, 18 Jan 2016 20:52:58"))]

,From,To,Date,Subject,Date_new,Datetime_utc,datetime_ist,only_Date,only_time
55105,CitiAlert.India@citicorp.com,[ghoshm21@gmail.com],"Mon, 18 Jan 2016 20:52:58 +0530 (GMT+05:30)",CitiAlert: Confirmation of transaction on your...,"Mon, 18 Jan 2016 20:52:58 +0530",2016-01-18 15:22:58+00:00,2016-01-18 20:52:58+05:30,2016-01-18,20:52:58
102010,CitiAlert.India@citicorp.com,[ghoshm21@gmail.com],"Mon, 18 Jan 2016 20:52:58 +0530 (GMT+05:30)",CitiAlert: Confirmation of transaction on your...,"Mon, 18 Jan 2016 20:52:58 +0530",2016-01-18 15:22:58+00:00,2016-01-18 20:52:58+05:30,2016-01-18,20:52:58


In [232]:
email_data[(email_data.Date.str.contains("(o'clock GMT.*)"))]

,From,To,Date,Subject,Date_new,Datetime_utc,datetime_ist,only_Date,only_time
1309,marketing@toshiba.eu,[ghoshm21@gmail.com],"Monday, 3 March 2008 17:54:18 o'clock GMT",Mobility Without Boundaries,"Monday, 3 March 2008 17:54:18 GMT",2008-03-03 17:54:18+00:00,2008-03-03 23:24:18+05:30,2008-03-03,23:24:18
43308,marketing@toshiba.eu,[ghoshm21@gmail.com],"Monday, 3 March 2008 17:54:18 o'clock GMT",Mobility Without Boundaries,"Monday, 3 March 2008 17:54:18 GMT",2008-03-03 17:54:18+00:00,2008-03-03 23:24:18+05:30,2008-03-03,23:24:18


In [208]:
# test = email_data[(email_data.Subject == 'User Successfully Added')]
test = email_data[(email_data.Subject == 'DSL Welcome Letter') | (email_data.Subject == 'User Successfully Added') | (email_data.Date.str.contains("\(GMT\+05:30\)"))]
test

,From,To,Date,Subject
0,admin@anandrathi.com,[GHOSHM21@GMAIL.COM],Fri Sep 14 13:44:44 2007 GMT,User Successfully Added
3,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 16:03:04 +0530",DSL Welcome Letter
4,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 15:59:35 +0530",DSL Welcome Letter
814,admin@anandrathi.com,[GHOSHM21@GMAIL.COM],Fri Sep 14 13:44:44 2007 GMT,User Successfully Added
3378,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 15:59:35 +0530",DSL Welcome Letter
3379,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 16:03:04 +0530",DSL Welcome Letter
42746,admin@anandrathi.com,[GHOSHM21@GMAIL.COM],Fri Sep 14 13:44:44 2007 GMT,User Successfully Added
46101,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 15:59:35 +0530",DSL Welcome Letter
46102,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 16:03:04 +0530",DSL Welcome Letter
92425,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 15:59:35 +0530",DSL Welcome Letter


In [113]:
# from datetime import datetime, timedelta
# remove the string ' (GMT+05:30)'
test['Date'] = test['Date'].str.replace("\(GMT\+05:30\)", "")
test['Datetime_utc'] = pd.to_datetime(test.Date, utc=True)
test['datetime_ist'] = test.Datetime_utc.dt.tz_convert('Asia/Kolkata')
test['only_Date'] = test['datetime_ist'].dt.strftime('%Y-%m-%d')
test['only_time'] = test['datetime_ist'].dt.strftime('%H:%M:%S')
test

,From,To,Date,Subject,Datetime_utc,datetime_ist,only_Date,only_time
0,admin@anandrathi.com,[GHOSHM21@GMAIL.COM],Fri Sep 14 13:44:44 2007 GMT,User Successfully Added,2007-09-14 13:44:44+00:00,2007-09-14 19:14:44+05:30,2007-09-14,19:14:44
3,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 16:03:04 +0530",DSL Welcome Letter,2011-02-15 10:33:04+00:00,2011-02-15 16:03:04+05:30,2011-02-15,16:03:04
4,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 15:59:35 +0530",DSL Welcome Letter,2011-02-15 10:29:35+00:00,2011-02-15 15:59:35+05:30,2011-02-15,15:59:35
5,CitiAlert.India@citicorp.com,[ghoshm21@gmail.com],"Sun, 13 Mar 2011 08:10:24 +0530",CITIBANK BILL PAY REGISTRATION,2011-03-13 02:40:24+00:00,2011-03-13 08:10:24+05:30,2011-03-13,08:10:24
7,CitiAlert.India@citicorp.com,[ghoshm21@gmail.com],"Fri, 22 Apr 2011 03:41:39 +0530",CITIBANK BILL PAY,2011-04-21 22:11:39+00:00,2011-04-22 03:41:39+05:30,2011-04-22,03:41:39
...,...,...,...,...,...,...,...,...
102006,CitiAlert.India@citicorp.com,[ghoshm21@gmail.com],"Sun, 13 Mar 2011 08:06:24 +0530",Bill Payment details,2011-03-13 02:36:24+00:00,2011-03-13 08:06:24+05:30,2011-03-13,08:06:24
102007,citialert.india@citicorp.com,[ghoshm21@gmail.com],"Fri, 13 Jul 2012 20:58:02 +0530",Citi Purchase Alert,2012-07-13 15:28:02+00:00,2012-07-13 20:58:02+05:30,2012-07-13,20:58:02
102008,citialert.india@citicorp.com,[ghoshm21@gmail.com],"Fri, 13 Jul 2012 20:03:38 +0530",Citi Purchase Alert,2012-07-13 14:33:38+00:00,2012-07-13 20:03:38+05:30,2012-07-13,20:03:38
102010,CitiAlert.India@citicorp.com,[ghoshm21@gmail.com],"Mon, 18 Jan 2016 20:52:58 +0530",CitiAlert: Confirmation of transaction on your...,2016-01-18 15:22:58+00:00,2016-01-18 20:52:58+05:30,2016-01-18,20:52:58


In [30]:
import pandas as pd
# date='Fri Sep 14 13:44:44 2007 GMT'
date = 'Tue, 15 Feb 2011 16:03:04 +0530'
# date_time=pd.to_datetime(date, format='%a %b %d %H:%M:%S %Y %Z')
date_time=pd.to_datetime(date, format='%a, %d %b  %Y %H:%M:%S %z')
date_time

Timestamp('2011-02-15 16:03:04+0530', tz='pytz.FixedOffset(330)')

In [38]:
test['new_date'] = pd.to_datetime(test['Date'], format='%a, %d %b  %Y %H:%M:%S %z', errors='ignore')
test

,From,To,Date,Subject,new_date
3,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 16:03:04 +0530",DSL Welcome Letter,2011-02-15 16:03:04+05:30
4,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 15:59:35 +0530",DSL Welcome Letter,2011-02-15 15:59:35+05:30
3378,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 15:59:35 +0530",DSL Welcome Letter,2011-02-15 15:59:35+05:30
3379,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 16:03:04 +0530",DSL Welcome Letter,2011-02-15 16:03:04+05:30
46101,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 15:59:35 +0530",DSL Welcome Letter,2011-02-15 15:59:35+05:30
46102,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 16:03:04 +0530",DSL Welcome Letter,2011-02-15 16:03:04+05:30
92425,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 15:59:35 +0530",DSL Welcome Letter,2011-02-15 15:59:35+05:30
92426,care.karnataka@airtel.in,[GHOSHM21@GMAIL.COM],"Tue, 15 Feb 2011 16:03:04 +0530",DSL Welcome Letter,2011-02-15 16:03:04+05:30
